In [1]:
#IMPORT STUFF RELATED TO DATA MANAGEMENT
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns
import regex as r

In [2]:
database_path = 'NEST_Pandemia.sqlite'

conn = sql.connect(database_path) # nest con filtro de pandemia
# conn = sql.connect('NEST_Pandemia.sqlite') # nest sin filtro de pandemia

# extrayendo placa y el año de la fecha de inicio de vigencia de la poliza
# este analisis es por placa, si se quisiera ver persistencia del cliente solo tendríamos que cambiar "numplaca" por "codcli"

query = "select DESCMARCA, DESCMODELO, SUB_AGRUPADOR, NUMPLACA, strftime('%Y', FECINIVALID ) ano from nest where tipoveh= 'P' and CODPOL in ('02B','02BR') and FECINIVALID > '2003-01-01' group by numplaca, ano, DESCMARCA, DESCMODELO, SUB_AGRUPADOR " 
basico = pd.read_sql_query(query, conn)

In [3]:
# limpieza de placas usando regex 

# ^ - This symbol anchors the pattern to the beginning of the string.
# (?: ...) - This is a non-capturing group, used to group parts of the pattern together.
# [A-H0-9]{1} - Match exactly one character that is either an uppercase letter from A to H or a digit from 0 to 9.
# [A-Z0-9]{1} - Match exactly one character that is either an uppercase letter from A to Z or a digit from 0 to 9.
# [0-9]{4} - Match exactly four digits (0 to 9).
# $ - This symbol anchors the pattern to the end of the string.

regex = r"^(?:[A-I0-9]{1}[A-Z0-9]{1}[0-9]{4})$"
basico.dropna(inplace=True)
basico = basico[basico.NUMPLACA.str.match(regex)]

In [4]:
basico.tail(50)

,DESCMARCA,DESCMODELO,SUB_AGRUPADOR,NUMPLACA,ano
956969,ACURA,MDX,SUV COMPACTO LUJO,EI0557,2023
956970,HONDA,CRV,CRV,EI0558,2023
956971,HONDA,HRV,MEDIANO,EI0559,2023
956972,HONDA,CRV,CRV,EI0561,2023
956973,HONDA,PILOT,SUV,EI0565,2023
956974,HONDA,CRV,CRV,EI0568,2023
956975,HONDA,PILOT,SUV,EI0569,2023
956976,HONDA,CRV,CRV,EI0570,2023
956977,ACURA,RDX,SUV COMPACTO LUJO,EI0571,2023
956978,ACURA,RDX,SUV COMPACTO LUJO,EI0572,2023


In [5]:
# creando binary table para crear el triangulo de persistencia

basico = basico.sort_values(by=['NUMPLACA', 'ano'], ascending=[True, True])
#basico['ano'] = pd.to_numeric(basico['ano'], errors='coerce') # Convert the 'ano' column to numeric (integer or float) data type
basico['ano'] = basico['ano'].astype(int)
basico['DETECTOR CORTE PLACA'] = (basico['NUMPLACA'] == basico['NUMPLACA'].shift()).astype(int)
basico['DETECTOR CORTE AÑO'] = (basico['ano'].diff() == 1).astype(int)
basico['COMBINADOR DE CORTES'] = (basico['DETECTOR CORTE PLACA'] + basico['DETECTOR CORTE AÑO'] == 2).astype(int)
basico.reset_index(drop=True, inplace=True)
basico.at[0, 'ACUMULADOR'] = 0

# Iterate through the 'basico' DataFrame to calculate cumulative values.

for i in range(len(basico)):
    if i == 0:
        pass
    else:
        if basico['COMBINADOR DE CORTES'][i] == 1:
            value = basico['COMBINADOR DE CORTES'][i] + basico['ACUMULADOR'][i-1]
        else: 
            value = 0
        basico.at[i, 'ACUMULADOR'] = value

basico['ACUMULADOR'] = basico['ACUMULADOR'].astype(int)

# creando binary table

#basico['ACUMULADOR'] = basico.apply(lambda row: row['COMBINADOR DE CORTES'] + row['ACUMULADOR'].shift() if row['COMBINADOR DE CORTES'] == 1 else 0, axis=1)
#basico['ACUMULADOR'] = basico['COMBINADOR DE CORTES'].eq(1).mul(basico['COMBINADOR DE CORTES'] + basico['ACUMULADOR'].shift(fill_value=0))
basico['PRESENTACION'] = basico['ACUMULADOR'] + 1
# Assuming your DataFrame is named 'basico', and the columns are 'AÑO INICIAL' (H), 'ano' (B), 'PRESENTACION' (G), and 'ACUMULADOR' (F)
basico['AÑO INICIAL'] = basico.apply(lambda row: row['ano'] if row['PRESENTACION'] == 1 else row['ano'] - row['ACUMULADOR'], axis=1)

In [6]:
# agregando key de marca modelo
basico["marca_modelo"] = basico["DESCMARCA"] + "_" + basico["DESCMODELO"]

In [7]:
basico.to_csv('basiconumplaca.csv', index=False)

In [8]:
# hacer pivot table
triangulo = basico.pivot_table(values="NUMPLACA", index="AÑO INICIAL", columns="ACUMULADOR", aggfunc="count")
#triangulo.reset_index(drop=True)
triangulo = pd.DataFrame(triangulo.to_records())

triangulo

,AÑO INICIAL,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,2003,9147.0,7192.0,5386.0,4055.0,3139.0,2426.0,1968.0,1597.0,1292.0,...,676.0,566.0,475.0,402.0,337.0,294.0,244.0,208.0,167.0,52.0
1,2004,3089.0,2417.0,1802.0,1324.0,1001.0,786.0,643.0,512.0,396.0,...,198.0,163.0,133.0,111.0,90.0,72.0,57.0,51.0,17.0,NaN
2,2005,3980.0,3425.0,2657.0,2139.0,1743.0,1420.0,1114.0,877.0,696.0,...,361.0,308.0,251.0,210.0,175.0,148.0,126.0,58.0,NaN,NaN
3,2006,2564.0,2066.0,1623.0,1319.0,1089.0,856.0,670.0,548.0,430.0,...,219.0,190.0,160.0,136.0,116.0,98.0,44.0,NaN,NaN,NaN
4,2007,6149.0,4376.0,3115.0,2437.0,1877.0,1446.0,1098.0,874.0,697.0,...,394.0,344.0,288.0,250.0,213.0,107.0,NaN,NaN,NaN,NaN
5,2008,5066.0,4067.0,2902.0,2184.0,1672.0,1186.0,929.0,712.0,583.0,...,334.0,277.0,225.0,171.0,57.0,NaN,NaN,NaN,NaN,NaN
6,2009,5090.0,4066.0,2968.0,2302.0,1795.0,1385.0,1077.0,858.0,695.0,...,423.0,364.0,307.0,113.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,15629.0,7341.0,2994.0,2326.0,1828.0,1429.0,1130.0,922.0,762.0,...,429.0,361.0,156.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011,13908.0,4582.0,3433.0,2678.0,2081.0,1694.0,1365.0,1099.0,922.0,...,576.0,264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,7304.0,5785.0,4300.0,3174.0,2473.0,2023.0,1681.0,1396.0,1149.0,...,369.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:

# hacer pivot table
triangulo = basico.pivot_table(values="NUMPLACA", index="AÑO INICIAL", columns="ACUMULADOR", aggfunc="count")
#triangulo.reset_index(drop=True)
triangulo = pd.DataFrame(triangulo.to_records())

In [10]:
# calcular probabilidades

columnas = triangulo.columns.tolist()[1:]
P_triangulo = pd.DataFrame(triangulo['AÑO INICIAL'])

for i in range(len(columnas)-1):  # Realizar 10 iteraciones para manejar todas las divisiones
    P_triangulo[str(i)] =  triangulo[str(i + 1)] / triangulo[str(i)] 
P_triangulo


# HASTA AQUÍ 

,AÑO INICIAL,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,2003,0.786269,0.748888,0.752878,0.774106,0.772858,0.811212,0.811484,0.809017,0.781734,...,0.830467,0.837278,0.839223,0.846316,0.838308,0.872404,0.829932,0.852459,0.802885,0.311377
1,2004,0.782454,0.745552,0.734739,0.756042,0.785215,0.818066,0.796267,0.773438,0.777778,...,0.838983,0.823232,0.815951,0.834586,0.810811,0.800000,0.791667,0.894737,0.333333,NaN
2,2005,0.860553,0.775766,0.805043,0.814867,0.814687,0.784507,0.787253,0.793615,0.798851,...,0.789934,0.853186,0.814935,0.836653,0.833333,0.845714,0.851351,0.460317,NaN,NaN
3,2006,0.805772,0.785576,0.812693,0.825625,0.786042,0.782710,0.817910,0.784672,0.797674,...,0.799270,0.867580,0.842105,0.850000,0.852941,0.844828,0.448980,NaN,NaN,NaN
4,2007,0.711660,0.711837,0.782343,0.770209,0.770378,0.759336,0.795993,0.797483,0.830703,...,0.822547,0.873096,0.837209,0.868056,0.852000,0.502347,NaN,NaN,NaN,NaN
5,2008,0.802803,0.713548,0.752584,0.765568,0.709330,0.783305,0.766416,0.818820,0.804460,...,0.847716,0.829341,0.812274,0.760000,0.333333,NaN,NaN,NaN,NaN,NaN
6,2009,0.798821,0.729956,0.775606,0.779757,0.771588,0.777617,0.796657,0.810023,0.848921,...,0.852823,0.860520,0.843407,0.368078,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,0.469704,0.407846,0.776887,0.785899,0.781729,0.790763,0.815929,0.826464,0.824147,...,0.841176,0.841492,0.432133,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011,0.329451,0.749236,0.780076,0.777072,0.814032,0.805785,0.805128,0.838944,0.843818,...,0.868778,0.458333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,0.792032,0.743302,0.738140,0.779143,0.818035,0.830944,0.830458,0.823066,0.855527,...,0.445115,0.002710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
